In [1]:
import requests
from textblob import TextBlob
from bs4 import BeautifulSoup

In [6]:
def get_news_articles(stock_ticker):
    sources = [
        ("https://www.moneycontrol.com/news/business/stocks/", "moneycontrol"),
        ("https://www.bloomberg.com/quote/" + stock_ticker + ":IN", "bloomberg"),
        ("https://www.reuters.com/finance/stocks/overview/" + stock_ticker, "reuters"),
        ("https://www.cnbc.com/quotes/" + stock_ticker, "cnbc")
    ]
    
    all_news_data = []
    
    for source, source_type in sources:
        response = requests.get(source)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            
            if source_type == "moneycontrol":
                articles = soup.find_all("li", class_="clearfix")
            elif source_type == "bloomberg":
                articles = soup.find_all("div", class_="story-story-grid-story__3MlKt")
            elif source_type == "reuters":
                articles = soup.find_all("div", class_="topStory")
            elif source_type == "cnbc":
                articles = soup.find_all("div", class_="QuotePage-summary")
            
            for article in articles:
                text = article.get_text() if source_type == "moneycontrol" else article.find("h1").get_text()
                all_news_data.append({"source": source_type, "headline": text})
    
    return all_news_data

In [7]:
def analyze_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    return sentiment


In [8]:
def get_final_sentiment(sentiments):
    positive_count = sum(1 for sentiment in sentiments if sentiment > 0)
    negative_count = sum(1 for sentiment in sentiments if sentiment < 0)
    
    if positive_count > negative_count:
        return "Positive"
    elif negative_count > positive_count:
        return "Negative"
    else:
        return "Neutral"


In [9]:
def get_stock_sentiment(stock_ticker):
    news_data = get_news_articles(stock_ticker)
    
    if news_data:
        sentiments = [analyze_sentiment(news["headline"]) for news in news_data]
        final_sentiment = get_final_sentiment(sentiments)
        
        result = {
            "ticker": stock_ticker,
            "sentiment": final_sentiment,
            "news": [{"source": news['source'], "headline": news['headline']} for news in news_data]
        }
    else:
        result = {"error": "Error fetching news articles."}

    return result

In [10]:
stock_ticker = input("Enter the stock ticker: ")
news_data = get_news_articles(stock_ticker)
  
if news_data:
      sentiments = [analyze_sentiment(news["headline"]) for news in news_data]
      final_sentiment = get_final_sentiment(sentiments)
      
      print(f"Final sentiment for {stock_ticker}: {final_sentiment}")
      
      for news in news_data:
          print(f"Source: {news['source']}")
          print(f"Headline: {news['headline']}")
          print(f"Sentiment: {analyze_sentiment(news['headline'])}")
          print("-" * 50)
else:
      print("Error fetching news articles.")

Final sentiment for tcs: Positive
Source: moneycontrol
Headline:    September 10, 2023 07:48 AM IST  Dalal Street Week Ahead | India, US inflation, crude prices, FII funds to determine D-St dynamics next week Even as domestic equity markets posted solid gains last week, foreign institutional investors sold India equities worth Rs 9,300 crore. Going ahead, this is a key monitorable 

Sentiment: -0.06562499999999999
--------------------------------------------------
Source: moneycontrol
Headline:   September 09, 2023 09:03 AM IST  Market extends gain on widespread buying but rupee takes a fall The equity benchmarks gained almost 2 percent for the week, with all sectoral indices ending in the green. The rupee remained under pressure to close the week at 82.94 against the dollar 

Sentiment: -0.2
--------------------------------------------------
Source: moneycontrol
Headline:   September 08, 2023 04:37 PM IST  Market Live: Nifty closes above 19,800; Sensex up 325 points |Bajar Gupshup Amo